In [11]:
from clickhouse_driver import Client

client = Client('localhost', port=9000)

client.execute('''CREATE DATABASE db''')

[]

> In the first showcase, a table is created which, when accessed, 
>
>gives you the coordinates of the `ISS` (International Space Station,) at the current moment:

In [12]:
client = Client('localhost', port=9000, database='db')

client.execute(
'''
create view mks as 
SELECT *
FROM url('http://api.open-notify.org/iss-now.json', JSONAsString, 'json String');
'''
)

[]

> The second dataset contains cities with their coordinates:

In [13]:
client.execute(
'''
create view cities as
SELECT *
FROM url(
    'https://raw.githubusercontent.com/dmitrii12334/clickhouse/main/cities',
    CSVWithNames,
    'region String, city String, latitude Float64, longitude Float64');
'''
)

[]

> The distance between the ISS (international space station) and the nearest 10 cities:

In [14]:
client.execute(
'''
select
    city,
    toFloat64(JSONExtractString(
    JSONExtractString(json, 'iss_position'), 'longitude')) as mks_longitude,
    toFloat64(JSONExtractString(
    JSONExtractString(json, 'iss_position'), 'latitude')) as mks_latitude,
    cities.longitude as city_long,
    cities.latitude as city_lat,
    greatCircleDistance(mks_longitude, mks_latitude, city_long, city_lat) / 1000 as distance
from mks, cities
order by distance
limit 10
'''
)

[("Mek'ele", 40.4797, 13.7663, 39.47528, 13.49667, 112.60484375),
 ('Ādīgrat', 40.4797, 13.7663, 39.462, 14.277, 123.60765625),
 ('Edd', 40.4797, 13.7663, 41.6938, 13.93088, 132.349265625),
 ('Maych’ew', 40.4797, 13.7663, 39.54222, 12.7875, 148.791984375),
 ('Adi Keyh', 40.4797, 13.7663, 39.37722, 14.84444, 168.7688125),
 ('Korem', 40.4797, 13.7663, 39.52278, 12.50583, 174.302609375),
 ('Axum', 40.4797, 13.7663, 38.72337, 14.12109, 193.60221875),
 ('Dek’emhāre', 40.4797, 13.7663, 39.0475, 15.07, 211.669625),
 ('Robīt', 40.4797, 13.7663, 39.63333, 12.01667, 215.09515625),
 ('Mendefera', 40.4797, 13.7663, 38.81528, 14.88722, 218.381828125)]